In [ ]:
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from platform import python_version
import os


# Создание pdf ноутбука и отправка его на почту



In [ ]:
def colab_pdf(file_name, notebookpath,path_to_save,download_=True):

    # Checking if file_name passed is a sring.
    if not isinstance(file_name, str):
        raise TypeError(
            f"expected a string as file_name, but got {type(file_name)} instead."
        )

    # Using the defaults used by google.colab
    drive_mount_point = "/content/drive/"
    gdrive_home = os.path.join(path_to_save)

    # If the drive is not already mounted, attempt to mount it.
    if not os.path.isdir(gdrive_home):
        from google.colab import drive

        drive.mount(drive_mount_point)

    # Check if the notebook exists in the Drive.
    if not os.path.isfile(os.path.join(notebookpath, file_name)):
        raise ValueError(f"file '{file_name}' not found in path '{notebookpath}'.")

    # Installing all the recommended packages.
    get_ipython().system(
        "apt update >> /dev/null && apt install texlive-xetex texlive-fonts-recommended texlive-generic-recommended >> /dev/null"
    )

    # If pdf with the same name exists, remove it.
    pdf_file = os.path.join(gdrive_home, file_name.split(".")[0] + ".pdf")
    
    if os.path.isfile(pdf_file):
        os.remove(pdf_file)

    # Attempt to convert to pdf and save it in Gdrive home dir using jupyter nbconvert command.
    try:
        get_ipython().system(
            "jupyter nbconvert --output-dir='$gdrive_home' '$notebookpath''$file_name' --to pdf "
        )
    except:
        return "nbconvert error"

    if download_:
      # Attempt to download the file to system.
      try:
          from google.colab import files

          file_name = file_name.split(".")[0] + ".pdf"
          files.download(gdrive_home + file_name)
      except:
          return "File Download Unsuccessful. Saved in Google Drive"

      return "File ready to be Downloaded and Saved to Drive"
      
colab_pdf('Transfer_learning.ipynb',notebookpath="/content/drive/My Drive/Study/ML/Assigment5/",path_to_save='/content/drive/My Drive/Study/ML/PDF_notebooks/',download_=False)

In [ ]:
def send_to_mail(user,password,recipients,subject,text,filepath=None):
  sender = user
  html = '<html><head></head><body><p>' + text + '</p></body></html>'

  msg = MIMEMultipart('alternative')
  msg['Subject'] = subject
  msg['From'] = sender
  msg['To'] = ', '.join(recipients)
  msg['Reply-To'] = sender
  msg['Return-Path'] = sender
  msg['X-Mailer'] = 'Python/' + (python_version())

  part_text = MIMEText(text, 'plain')
  part_html = MIMEText(html, 'html')
  try:
    basename = os.path.basename(filepath)
    filesize = os.path.getsize(filepath)

    part_text = MIMEText(text, 'plain')
    part_html = MIMEText(html, 'html')
    part_file = MIMEBase('application', 'octet-stream; name="{}"'.format(basename))
    part_file.set_payload(open(filepath, "rb").read())
    part_file.add_header('Content-Description', basename)
    part_file.add_header('Content-Disposition', 'attachment; filename="{}"; size={}'.format(basename, filesize))
    encoders.encode_base64(part_file)

    msg.attach(part_text)
    msg.attach(part_html)
    msg.attach(part_file)
  except:
    pass

  mail = smtplib.SMTP_SSL(server)
  mail.login(user, password)
  mail.sendmail(sender, recipients, msg.as_string())
  mail.quit()

In [ ]:
def create_and_send(notebook_name,notebookpath,path_to_save,download_,user,password,recipients,subject,text,filepath):
  colab_pdf(notebook_name,notebookpath,path_to_save,download_)
  send_to_mail(user,password,recipients,subject,text,filepath)

notebook_name='_.ipynb'
notebookpath="путь к ноутбуку"
path_to_save='путь, куда сохранить ноутбук'
download_=False # Нужно ли скачивать ноутбук

user = 'Логин'
password = 'Пароль'
recipients = ['Кому_отправить']
subject = 'Тема'
text = 'Сам_текст_сообщения'
filepath='Файл, который нужно прекрепить'